# Scenario

The whole world seems to be hearing about your new amazing abilities to analyze big data and build useful systems for them. You have just taken up a new contract with online food delivery company. This company is trying to differentiate iteself by recommending new meals to customers based off  other customers likings.

Can you build them a recommendation system?

Your final result should be in the form of a function that can take in a Spark DataFrame of a single customer's ratings for various meals and output their top 3 suggested meals.  For example:


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('movielens_ratings.csv')

In [3]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0


In [4]:
df.corr()

,movieId,rating,userId
movieId,1.000000,0.036569,0.003267
rating,0.036569,1.000000,0.056411
userId,0.003267,0.056411,1.000000


In [5]:
import numpy as np

In [19]:
df['mealskew'] = df['movieId'].apply(lambda id: 0 if id > 31 else id)

In [20]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0
mealskew,1501.0,5.019320,8.962159,0.0,0.0,0.0,6.0,31.0


In [21]:
mealmap = { 2. : "Chicken Curry",   
           3. : "Spicy Chicken Nuggest",   
           5. : "Hamburger",   
           9. : "Taco Surprise",  
           11. : "Meatloaf",  
           12. : "Ceaser Salad",  
           15. : "BBQ Ribs",  
           17. : "Sushi Plate",  
           19. : "Cheesesteak Sandwhich",  
           21. : "Lasagna",  
           23. : "Orange Chicken",
           26. : "Spicy Beef Plate",  
           27. : "Salmon with Mashed Potatoes",  
           28. : "Penne Tomatoe Pasta",  
           29. : "Pork Sliders",  
           30. : "Vietnamese Sandwich",  
           31. : "Chicken Wrap",  
           np.nan: "Cowboy Burger",   
           4. : "Pretzels and Cheese Plate",   
           6. : "Spicy Pork Sliders",  
           13. : "Mandarin Chicken PLate",  
           14. : "Kung Pao Chicken",
           16. : "Fried Rice Plate",  
           8. : "Chicken Chow Mein",  
           10. : "Roasted Eggplant ",  
           18. : "Pepperoni Pizza",  
           22. : "Pulled Pork Plate",   
           0. : "Cheese Pizza",   
           1. : "Burrito",   
           7. : "Nachos",  
           24. : "Chili",  
           20. : "Southwest Salad",  
           25.: "Roast Beef Sandwich"}

In [22]:
df['meal_name'] = df['mealskew'].map(mealmap)

In [23]:
df.to_csv('Meal_Info.csv',index=False)

In [24]:
from pyspark.sql import SparkSession

In [25]:
spark = SparkSession.builder.appName('recconsulting').getOrCreate()

In [26]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [27]:
data = spark.read.csv('Meal_Info.csv',inferSchema=True,header=True)

In [28]:
data.head()

Row(movieId=2, rating=3.0, userId=0, mealskew=2, meal_name='Chicken Curry')

In [29]:
(training, test) = data.randomSplit([0.8, 0.2])

In [32]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="mealskew", ratingCol="rating")
model = als.fit(training)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

predictions.show()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+-------+------+------+--------+--------------------+-----------+
|movieId|rating|userId|mealskew|           meal_name| prediction|
+-------+------+------+--------+--------------------+-----------+
|     31|   1.0|    13|      31|        Chicken Wrap|  0.6775918|
|     31|   3.0|     8|      31|        Chicken Wrap|   3.087021|
|     31|   1.0|    29|      31|        Chicken Wrap|  1.3616024|
|     31|   1.0|     0|      31|        Chicken Wrap|  1.3988154|
|     28|   3.0|     1|      28| Penne Tomatoe Pasta|  1.0993308|
|     28|   2.0|    15|      28| Penne Tomatoe Pasta|   0.966365|
|     28|   1.0|    10|      28| Penne Tomatoe Pasta|  1.0642205|
|     28|   1.0|     2|      28| Penne Tomatoe Pasta|   3.886753|
|     28|   1.0|     0|      28| Penne Tomatoe Pasta|  1.6128743|
|     26|   1.0|    22|      26|    Spicy Beef Plate|-0.62599796|
|     26|   1.0|    13|      26|    Spicy Beef Plate|0.024555467|
|     26|   1.0|     6|      26|    Spicy Beef Plate|  1.3958986|
|     26| 